In [35]:
import pandas as pd
import numpy as np

In [36]:
data = pd.read_csv("./data/train_dataset.csv")

data['is_test'] = False
evaluation = pd.read_csv("./data/evaluation_public.csv")
evaluation['is_test'] = True
sample = pd.read_csv("./data/sample_submission.csv")

all_data = pd.concat([data, evaluation]).reset_index(drop=True)

# 特征处理

In [37]:
all_data['date'] = pd.to_datetime(all_data['time'])
all_data['hour'] = all_data['date'].dt.hour
all_data['year'] = all_data['date'].dt.year
all_data['month'] = all_data['date'].dt.month
all_data['minute'] = all_data['date'].dt.minute
all_data['weekday'] = all_data['date'].dt.weekday
all_data['day'] = all_data['date'].dt.day
all_data['hour'] = all_data['date'].dt.hour
all_data['ts'] = all_data['hour']*3600 + all_data['minute']*60 + all_data['date'].dt.second

In [38]:
# year','month','day','hour',
features = [ 'JS_NH3', 'CS_NH3', 'JS_TN', 'CS_TN', 'JS_LL', 'CS_LL', 'MCCS_NH4', 'MCCS_NO3', 'JS_COD', 'CS_COD', 'JS_SW', 'CS_SW', 'B_HYC_NH4', 'B_HYC_XD', 'B_HYC_MLSS', 'B_HYC_JS_DO', 'B_HYC_DO', 'B_CS_MQ_SSLL', 'B_QY_ORP', 'N_HYC_NH4', 'N_HYC_XD', 'N_HYC_MLSS', 'N_HYC_JS_DO', 'N_HYC_DO', 'N_CS_MQ_SSLL', 'N_QY_ORP','weekday','hour', 'ts']
features = [f for f in features if f not in ['time', 'Label1', 'Label2','Label1_log','Label2_log','CS_LL',
                                              'B_QY_ORP','JS_TN', 'CS_SW','MCCS_NH4','N_HYC_JS_DO','MCCS_NO3','JS_SW',
                                             ]]

labels = ['Label1', 'Label2']
train = all_data[~all_data['is_test']].copy(deep=True)
test = all_data[all_data['is_test']].copy(deep=True)
train = train.dropna(subset=['Label1', 'Label2']).reset_index(drop=True)
test['is_train'] = False
train['is_train'] = True
data = pd.concat([train, test]).reset_index(drop=True)

In [39]:
# import matplotlib.pyplot as plt
# for x in features:
#     if x not in ['weekday','hour', 'ts']:
#         plt.figure(1, figsize=(4, 10))
#         data[x][-30000:].plot()
#         plt.show()
#         plt.close()


In [40]:
df_list = []
i = 0
add_featuers = []
length = 0
for f in ['JS_NH3', 'CS_NH3', 'CS_TN', 'JS_LL', 'JS_COD', 'CS_COD', 'B_HYC_NH4', 'B_HYC_XD', 'B_HYC_MLSS', 'B_HYC_JS_DO', 'B_HYC_DO', 'B_CS_MQ_SSLL',
          'N_HYC_NH4', 'N_HYC_XD', 'N_HYC_MLSS', 'N_HYC_DO', 'N_CS_MQ_SSLL', 'N_QY_ORP']:
    for r in [7]:
        train_rolling = data[f].rolling(window=r, center=True)
        f_mean_name = 'rolling{}_{}_mean'.format(r,f)
        f_max_name = 'rolling{}_{}_max'.format(r,f)
        f_min_name = 'rolling{}_{}_min'.format(r,f)
        f_std_name = 'rolling{}_{}_std'.format(r,f)
        data[f_mean_name] = train_rolling.mean().fillna(0).values
        data[f_max_name] = train_rolling.max().fillna(0).values
        data[f_min_name] = train_rolling.min().fillna(0).values
        data[f_std_name] = train_rolling.std().fillna(0).values
        data[f'{f}_{f_mean_name}_cha'] = data[f] - data[f_mean_name]
        data[f'{f}_{f_max_name}_cha'] = data[f] - data[f_max_name]
        data[f'{f}_{f_min_name}_cha'] = data[f] - data[f_min_name]
        if i == 0:
            add_featuers.append(f_mean_name)
            add_featuers.append(f_max_name)
            add_featuers.append(f_min_name)
            add_featuers.append(f_std_name)
            add_featuers.append(f'{f}_{f_mean_name}_cha')
            add_featuers.append(f'{f}_{f_max_name}_cha')
            add_featuers.append(f'{f}_{f_min_name}_cha')
features.extend(add_featuers)

# 对所有的特征进行划分
for f in features:
    if f not in ['weekday','hour', 'ts']:
        data[f] = pd.qcut(data[f], q=100, labels=False, duplicates="drop")

train = data[data['is_train']].reset_index(drop=True)
test = data[~data['is_train']].reset_index(drop=True)

C:\Users\38125\AppData\Local\Temp/ipykernel_67520/1297101366.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{f}_{f_mean_name}_cha'] = data[f] - data[f_mean_name]
C:\Users\38125\AppData\Local\Temp/ipykernel_67520/1297101366.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{f}_{f_max_name}_cha'] = data[f] - data[f_max_name]
C:\Users\38125\AppData\Local\Temp/ipykernel_67520/1297101366.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times

In [41]:
train = train.dropna(subset=['Label1', 'Label2']).reset_index(drop=True)

In [42]:
train

,time,JS_NH3,CS_NH3,JS_TN,CS_TN,JS_LL,CS_LL,MCCS_NH4,MCCS_NO3,JS_COD,...,N_CS_MQ_SSLL_rolling7_N_CS_MQ_SSLL_mean_cha,N_CS_MQ_SSLL_rolling7_N_CS_MQ_SSLL_max_cha,N_CS_MQ_SSLL_rolling7_N_CS_MQ_SSLL_min_cha,rolling7_N_QY_ORP_mean,rolling7_N_QY_ORP_max,rolling7_N_QY_ORP_min,rolling7_N_QY_ORP_std,N_QY_ORP_rolling7_N_QY_ORP_mean_cha,N_QY_ORP_rolling7_N_QY_ORP_max_cha,N_QY_ORP_rolling7_N_QY_ORP_min_cha
0,2022/4/30 16:00,99.0,92.0,52.8394,77.0,27.0,2204.6711,0.0010,8.5796,99.0,...,98.0,85.0,84.0,0,0,0,0,93.0,49.0,50.0
1,2022/5/2 16:00,50.0,92.0,40.1798,49.0,80.0,1410.1064,0.1977,8.1337,59.0,...,98.0,85.0,84.0,0,0,0,0,93.0,49.0,50.0
2,2022/5/3 16:00,69.0,91.0,46.6933,78.0,20.0,2112.2976,0.1978,8.3301,70.0,...,97.0,84.0,83.0,0,0,0,0,93.0,49.0,50.0
3,2022/5/4 16:00,91.0,86.0,52.6615,76.0,61.0,875.0145,0.2086,8.3724,90.0,...,83.0,81.0,45.0,7,24,3,94,0.0,0.0,50.0
4,2022/5/5 16:00,77.0,73.0,48.7988,62.0,34.0,1495.3458,0.2059,7.4816,87.0,...,28.0,34.0,0.0,9,24,4,94,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35063,2022/7/18 2:30,49.0,60.0,30.2603,27.0,8.0,1111.9569,0.1428,9.0365,30.0,...,19.0,17.0,7.0,63,63,63,81,80.0,20.0,40.0
35064,2022/7/18 2:32,49.0,60.0,30.2603,27.0,10.0,1680.4688,0.1425,9.0495,30.0,...,43.0,45.0,15.0,63,63,62,82,19.0,11.0,34.0
35065,2022/7/18 2:34,49.0,60.0,30.2603,27.0,8.0,1633.5309,0.1425,9.0419,30.0,...,73.0,73.0,37.0,62,63,62,82,22.0,10.0,34.0
35066,2022/7/18 2:36,49.0,60.0,30.2603,27.0,8.0,1247.8890,0.1425,9.0093,30.0,...,39.0,49.0,9.0,62,62,61,80,23.0,9.0,32.0


# 删除分布不均衡的特征

In [43]:
train['test'] = 0
test['test'] = 1


In [44]:
import lightgbm as lgb
score_list = []
for label in labels:
    train[label] = train[label].fillna(0)
    tmp = train[~train[label].isna()].reset_index(drop=True).copy(deep=True)
    print(len(tmp))
    print(np.max(tmp[label]), np.min(tmp[label]))
    test_size = 4000
    X_train = tmp[features][:-test_size]
    X_test = tmp[features][-test_size:]
    y_train = tmp[label][:-test_size]
    y_test = tmp[label][-test_size:]

    model = lgb.LGBMRegressor(
        boosting="gbdt",
        max_depth=4,
        learning_rate=0.01,
        n_estimators=10000,
        min_child_weight=1,
        min_data_in_leaf=60,
        subsample = 0.7,
        feature_fraction=0.9,
        bagging_seed=1,
        reg_alpha=1,
        reg_lambda=1,  # 此处不改了
        min_sum_hessian_in_leaf=0.01,
        random_state=1212
    )

    model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], eval_metric=['mae'],
          early_stopping_rounds=150, verbose=200)

    model2 = lgb.LGBMRegressor(
        boosting="gbdt",
        max_depth=4,
        learning_rate=0.01,
        n_estimators=int(1.2*model.best_iteration_),
        min_child_weight=1,
        min_data_in_leaf=60,
        subsample = 0.7,
        feature_fraction=0.9,
        bagging_seed=1,
        reg_alpha=1,
        reg_lambda=1,  # 此处不改了
        min_sum_hessian_in_leaf=0.01,
        random_state=1212
    )
    model2.fit(tmp[features], tmp[label])
    score_list.append(np.sqrt(model.best_score_['valid_1']['l2']))
    test[label] = model2.predict(test[features])

35068
66326.05 814.49
Training until validation scores don't improve for 150 rounds
[200]	training's l1: 1509.31	training's l2: 4.75368e+06	valid_1's l1: 1271.9	valid_1's l2: 2.72246e+06
[400]	training's l1: 1213.14	training's l2: 3.22128e+06	valid_1's l1: 1233.02	valid_1's l2: 2.60666e+06
Early stopping, best iteration is:
[359]	training's l1: 1249.56	training's l2: 3.37592e+06	valid_1's l1: 1230.24	valid_1's l2: 2.59327e+06
35068
49464.13 719.71
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] min_data_in_leaf is set=60, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=60
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.01, min_child_weight=1 will be ignored. Current value: min_sum_hessian_in_leaf=0.01
Training until validation scores don't improve for 150 r

In [45]:
loss = np.mean(score_list)
1000/(1+loss)

0.7032271676086042

In [46]:
test[['time'] + labels].to_csv("./res/base_line.csv", index=False)